## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Предобработка данных не нужна.

Необходимо определить модель с максимально большим значением accuracy и долей правильных ответов не менее 75%.

## Описание данных

- сalls — количество звонков
- minutes — суммарная длительность звонков в минутах
- messages — количество sms-сообщений
- mb_used — израсходованный интернет-трафик в Мб
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [1]:
# Подключим все необходимые библиотеки:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

## Изучение данных

In [2]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('D:/users_behavior.csv')

In [3]:
display(data.head())
data.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [15]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [18]:
data.duplicated().sum()

0

Пропусков и явных дубликатов не обнаружено. Целевой признак выборки является столбик "is_ultra" и представлен категорией из 0 и 1 - это задача бинарной классификации.

## Создание выборок

Отделим 60% генеральной выборки для обучения модели, 20% для валидации и 20% для окончательной проверки дееспесобности выбранной модели. 

In [5]:
# В признаки модели (features) отнесём все данные выборки, за исключением столбца "is_ultra", 
# т.к. это целевой признак (target).

features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [6]:
# С помощью метода train_test_split разделим генеральную выбрку на тренировочную (*_train), валидационную (*_valid) 
# и тестовую (*_test)

features_train, features_rest, target_train, target_rest = \
    train_test_split(features, target, test_size=0.4, random_state=0)

features_valid, features_test, target_valid, target_test = \
    train_test_split(features_rest, target_rest, test_size=0.5, random_state=0)

Убедимся, что доли созданных выборок соответствуют необходимому нам соотношению 3:1:1.

In [26]:
display(f'Тренировочная выборка составила {round(features_train.shape[0] / data.shape[0] * 100)}% от генеральной')
display(f'Валидационная выборка составила {round(features_valid.shape[0] / data.shape[0] * 100)}% от генеральной')
display(f'Тестовая выборка составила {round(features_test.shape[0] / data.shape[0] * 100)}% от генеральной')

'Тренировочная выборка составила 60% от генеральной'

'Валидационная выборка составила 20% от генеральной'

'Тестовая выборка составила 20% от генеральной'

## Подбор модели 

### "Дерево решений"

In [7]:
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=0, max_depth=depth) 
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_result = result
        best_depth = depth

display('Точность модели "Дерево решений": {:.1f}'.format(best_result*100),
        "Лучшая глубина: {}".format(best_depth))

'Точность модели "Дерево решений": 81.2'

'Лучшая глубина: 7'

Перебрав циклом глубину от 1 до 10, определили, что лучшим гипер-параметром глубины является 7. Повторим этот же цикл, изменив гипер-параметр "criterion" с Jinny, который установлен по умолчанию, на "entropy" и изучимм результаты:

In [8]:
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=0, max_depth=depth, criterion='entropy') 
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_result = result
        best_depth = depth

display('Точность модели "Дерево решений" с критерием энтропии: {:.1f}'.format(best_result*100), 
        "Лучшая глубина: {}".format(best_depth))

'Точность модели "Дерево решений" с критерием энтропии: 80.9'

'Лучшая глубина: 4'

Лучшая глубина изменилась с 7 до 4, однако на максимальную точность модели это не оказало влияния.

### "Случайный лес"

In [9]:
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 41, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # обучим модель на тренировочной выборке
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth

display('Наибольшая точность модели "Случайный лес": {:.1f}'.format(best_result*100), 
      "При количестве деревьев: {}".format(best_est), 
      "И глубине: {}".format(best_depth))

'Наибольшая точность модели "Случайный лес": 82.1'

'При количестве деревьев: 20'

'И глубине: 11'

### "Ближайшие соседи"

In [10]:
best_result = 0
best_neigbors = 0
for neighbors in range(1, 11):
    model = KNeighborsClassifier(n_neighbors=neighbors) 
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_result = result
        best_neigbors = neighbors

display('Точность модели "Ближайшие соседи": {:.1f}'.format(best_result*100), 
        "Лучшее количество соседей: {}".format(best_neigbors))

/opt/conda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/opt/conda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to Tr

'Точность модели "Ближайшие соседи": 77.4'

'Лучшая глубина: 9'

Наилучшие результаты показала модель **"Случайный лес"** со следующими параметрами: 
- **кол-во деревьвев: 20;**
- **глубина: 11.**

Проверим точность выбранной модели на тестовой выборке с наиболее результативными гипер-параметрами.

## Проверка модели на тестовой выборке

In [11]:
model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=11)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
display('Точность модели "Случайный лес" на тестовой выборке: {:.1f} %'.format(result*100))

'Точность модели "Случайный лес" на тестовой выборке: 77.9 %'

Выбранная нами модель показала меньшую точность на тестовой выборке, однако достаточную, чтобы считать модель "Случайный лес" подходящей для прогнозирования. С целью самопроверки изучим точность остальных моделей на тестовой выборке:

In [12]:
model = DecisionTreeClassifier(random_state=0, max_depth=4) 
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
display('Точность модели "Дерево решений" на тестовой выборке: {:.1f} %'.format(result*100))

'Точность модели "Дерево решений" на тестовой выборке: 76.0 %'

In [13]:
model = KNeighborsClassifier(n_neighbors=9) 
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
display('Точность модели "Ближайшие соседи" на тестовой выборке: {:.1f} %'.format(result*100))

/opt/conda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


'Точность модели "Ближайшие соседи" на тестовой выборке: 75.9 %'

Эмперически доказано, что наилучшей моделью для прогнозированния тарифа пользователей мобильной сети "Мобилайн" является "Случайный лес".

## Проверка на адекватность

С помощью метода DummyClassifier, который делает прогноз, игнорируя входящие параметры, изучим модель "Случайный лес" на адекватность:

In [14]:
model_adequacy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
predictions = model_adequacy.predict(features_test)
result = accuracy_score(target_test, predictions)

print('Поверка адекватности: {:.1f} %'.format(result*100))

Поверка адекватности: 68.6 %


Выбранная модель "Случайный лес" прошла проверку на адекватность методом DummyClassifier, показав реультат 68.6%.